In [57]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm # Displays a progress bar
from math import sqrt
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchsummary import summary
from torchvision import datasets, io, models, ops, transforms, utils
from torch.utils.data import Dataset, Subset, DataLoader, random_split
import pandas as pd
from PIL import Image
# from torchvision import datasets, io, models, ops, transforms, utils
import os
# import data as dataset


In [58]:
if torch.cuda.is_available():
    print("Using the GPU. You are good to go!")
    device = 'cuda'
else:
    print("Using the CPU. Overall speed may be slowed down")
    device = 'cpu'

Using the CPU. Overall speed may be slowed down


In [59]:
df = pd.read_csv('output.csv', delimiter=';')

In [118]:
# df[df['partition']=='dev']
df[df['filename']=='deafvideo_4/mattref2005_5169']

,Unnamed: 0,filename,url,start_time,number_of_frames,width,height,label_proc,label_raw,label_notes,partition,signer,Label,start_l,end_l
23,1800,deafvideo_4/mattref2005_5169,http://www.deafvideo.tv/805784,0:03:20.420000,6,640,360,iat,iat,iat?,train,37,a,2,3


In [102]:
def file_exists(filename):
    folders = ['avg_dev', 'avg_test', 'avg_train']
    filename=filename.replace('/','_')+".png"
    print(filename)
    for folder in folders:
        if os.path.exists(os.path.join(folder, filename)):
            return True
    return False
class HandSignDataset(Dataset):
    def __init__(self, csv_file, root_dir, partition, transform=None):
        self.df = pd.read_csv(csv_file, delimiter=';')
        self.df = self.df[self.df['partition'] == partition]
    
        self.df=self.df[self.df['filename'].apply(file_exists)]

    # define a function to check if a file exists in any of the folders

        self.root_dir = root_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        filename = self.df.iloc[idx, self.df.columns.get_loc('filename')]
        filename_img = self.df.iloc[idx, self.df.columns.get_loc('filename')].replace('/','_')
        label = self.df.iloc[idx, self.df.columns.get_loc('Label')]
        

        image_path = os.path.join(self.root_dir, filename_img+".png")
        bbox_path = os.path.join("BBox", filename, "0000.txt")
        
        try:
            with open(bbox_path) as f:
                bbox_info = f.readline().split(',')
                print(bbox_info)
            x0, y0, x1, y1, _ = bbox_info
            x0, y0, x1, y1 = int(x0), int(y0), int(x1), int(y1)

            image = Image.open(image_path).convert('RGB')
            image = image.crop((x0, y0, x1, y1))
        except FileNotFoundError:
            image = Image.open(image_path).convert('RGB')
        if(np.sum(image)==0):
            print('ALL ZERO')

        if self.transform:
            image = self.transform(image)
        print('image: ',filename_img,"type and image:",image.shape,image)
    
        print('label: ',label)
#         utils.save_image(img, f"/ImageOutput/{filename_img}_T.png")
        # img1 = img1.numpy() # TypeError: tensor or list of tensors expected, got <class 'numpy.ndarray'>
#         save_image(img, filename +'_T.png')
        return image, label

In [112]:
# Define the transformation(s) to be applied to the images
transform = transforms.Compose([
    transforms.Resize((360, 640)),
    transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                          std=[0.229, 0.224, 0.225])
])

# Call the create_dataset function to create a PyTorch dataset
test_dataset = HandSignDataset(csv_file='output.csv', root_dir='avg_test', partition='test',transform=transform)
train_dataset = HandSignDataset(csv_file='output.csv', root_dir='avg_train', partition='train',transform=transform)
val_dataset = HandSignDataset(csv_file='output.csv', root_dir='avg_dev', partition='dev',transform=transform)


youtube_5_roberta_cordano_6136.png
youtube_5_roberta_cordano_6149.png
youtube_5_roberta_cordano_6154.png
youtube_6_roberta_cordano_6477.png
youtube_6_roberta_cordano_6480.png
deafvideo_3_kathyfans_3459.png
deafvideo_3_kathyfans_3473.png
deafvideo_3_kazio63_3504.png
deafvideo_3_kazio63_3505.png
deafvideo_3_kazio63_3507.png
deafvideo_3_kazio63_3508.png
youtube_1_joel_garcia_2863.png
youtube_2_nick_martin_3419.png
youtube_2_nick_martin_3435.png
youtube_5_david_o._reynolds_5819.png
youtube_5_keith_nolan_6011.png
youtube_5_keith_nolan_6012.png
deafvideo_4_floridaaahh_5724.png
deafvideo_4_floridaaahh_5725.png
misc_2_genie_gertz_2892.png
aslized_joseph_hill_1761.png
aslized_mj_bienvenu_7057.png
aslized_mj_bienvenu_7068.png
aslthat_joseph_wheeler_0589.png
deafvideo_1_taylerade_1231.png
deafvideo_1_taylerade_1275.png
deafvideo_1_wesley_arey_0913.png
deafvideo_1_wesley_arey_0915.png
deafvideo_2_dsport06_1618.png
deafvideo_2_dsport06_1625.png
deafvideo_2_dsport06_1627.png
deafvideo_3_deafaynrand_

In [113]:
batch_size=32
# Create a data loader for the dataset

# trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, collate_fn=custom_collate)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)



In [114]:
for batch in trainloader:
    img=batch[0]
    label=batch[1]

image:  deafvideo_2_dsport06_1625 type and image: torch.Size([3, 360, 640]) tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])
label:  a
image:  youtube_4_evelina_gaina_4869 type and image: torch.Size([3, 360, 640]) tensor([[[0.4314, 0.4314, 0.4314,  ..., 0.2980, 0

image:  misc_1_carol_padden_1911 type and image: torch.Size([3, 360, 640]) tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])
label:  a
image:  deafvideo_4_mattref2005_5199 type and image: torch.Size([3, 360, 640]) tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         

image:  youtube_3_raymond_merritt_4774 type and image: torch.Size([3, 360, 640]) tensor([[[0.5294, 0.5294, 0.5294,  ..., 0.3608, 0.3647, 0.3686],
         [0.5294, 0.5294, 0.5294,  ..., 0.3608, 0.3647, 0.3686],
         [0.5333, 0.5333, 0.5333,  ..., 0.3647, 0.3647, 0.3686],
         ...,
         [0.6549, 0.6588, 0.6549,  ..., 0.4039, 0.4392, 0.4706],
         [0.6627, 0.6627, 0.6471,  ..., 0.4667, 0.4784, 0.4902],
         [0.6627, 0.6588, 0.6510,  ..., 0.4196, 0.4196, 0.4902]],

        [[0.3647, 0.3647, 0.3647,  ..., 0.3059, 0.3098, 0.3137],
         [0.3647, 0.3647, 0.3647,  ..., 0.3059, 0.3098, 0.3137],
         [0.3686, 0.3686, 0.3686,  ..., 0.3098, 0.3098, 0.3137],
         ...,
         [0.6510, 0.6510, 0.6235,  ..., 0.2627, 0.2941, 0.3294],
         [0.6549, 0.6392, 0.6078,  ..., 0.3059, 0.3176, 0.3255],
         [0.6471, 0.6275, 0.6078,  ..., 0.3137, 0.3137, 0.3098]],

        [[0.2157, 0.2157, 0.2157,  ..., 0.2627, 0.2667, 0.2706],
         [0.2157, 0.2157, 0.2157,  ..., 0.

image:  youtube_5_ryan_lane_5874 type and image: torch.Size([3, 360, 640]) tensor([[[0.1294, 0.1333, 0.1333,  ..., 0.1137, 0.1137, 0.1137],
         [0.1333, 0.1373, 0.1373,  ..., 0.1137, 0.1137, 0.1137],
         [0.1412, 0.1412, 0.1412,  ..., 0.1137, 0.1137, 0.1137],
         ...,
         [0.0157, 0.0157, 0.0157,  ..., 0.0118, 0.0314, 0.0824],
         [0.0196, 0.0196, 0.0196,  ..., 0.0118, 0.0392, 0.1020],
         [0.0196, 0.0196, 0.0196,  ..., 0.0118, 0.0431, 0.1137]],

        [[0.0588, 0.0627, 0.0627,  ..., 0.0510, 0.0510, 0.0510],
         [0.0627, 0.0667, 0.0667,  ..., 0.0510, 0.0510, 0.0510],
         [0.0667, 0.0706, 0.0706,  ..., 0.0510, 0.0510, 0.0510],
         ...,
         [0.0157, 0.0157, 0.0157,  ..., 0.0078, 0.0353, 0.0863],
         [0.0196, 0.0196, 0.0196,  ..., 0.0078, 0.0392, 0.1059],
         [0.0196, 0.0196, 0.0196,  ..., 0.0039, 0.0431, 0.1176]],

        [[0.0588, 0.0627, 0.0627,  ..., 0.0510, 0.0510, 0.0510],
         [0.0627, 0.0667, 0.0667,  ..., 0.0510, 

image:  youtube_1_jackie_roth_3003 type and image: torch.Size([3, 360, 640]) tensor([[[0.0706, 0.0706, 0.0667,  ..., 0.0275, 0.0275, 0.0275],
         [0.0706, 0.0706, 0.0667,  ..., 0.0275, 0.0275, 0.0275],
         [0.0706, 0.0706, 0.0667,  ..., 0.0275, 0.0275, 0.0275],
         ...,
         [0.3882, 0.3882, 0.3882,  ..., 0.0902, 0.0863, 0.0863],
         [0.3843, 0.3843, 0.3843,  ..., 0.0863, 0.0863, 0.0863],
         [0.3843, 0.3843, 0.3843,  ..., 0.0863, 0.0863, 0.0863]],

        [[0.0314, 0.0314, 0.0275,  ..., 0.0039, 0.0039, 0.0039],
         [0.0314, 0.0314, 0.0275,  ..., 0.0039, 0.0039, 0.0039],
         [0.0314, 0.0314, 0.0275,  ..., 0.0039, 0.0039, 0.0039],
         ...,
         [0.4510, 0.4510, 0.4510,  ..., 0.0667, 0.0627, 0.0627],
         [0.4471, 0.4471, 0.4471,  ..., 0.0627, 0.0627, 0.0627],
         [0.4471, 0.4471, 0.4471,  ..., 0.0627, 0.0627, 0.0627]],

        [[0.0627, 0.0627, 0.0588,  ..., 0.0196, 0.0196, 0.0196],
         [0.0627, 0.0627, 0.0588,  ..., 0.0196

image:  misc_1_carol_padden_2190 type and image: torch.Size([3, 360, 640]) tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])
label:  a
image:  youtube_4_raymond_merritt_4841 type and image: torch.Size([3, 360, 640]) tensor([[[0.5294, 0.5294, 0.5294,  ..., 0.3608, 

image:  misc_1_carol_padden_2002 type and image: torch.Size([3, 360, 640]) tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])
label:  a
image:  youtube_6_tom_humphries_6656 type and image: torch.Size([3, 360, 640]) tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.

image:  aslized_felicia_williams_7040 type and image: torch.Size([3, 360, 640]) tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])
label:  a
image:  misc_2_aidan_mack_3183 type and image: torch.Size([3, 360, 640]) tensor([[[0.3412, 0.3216, 0.2784,  ..., 0.6863, 0.7

image:  misc_1_carol_padden_1974 type and image: torch.Size([3, 360, 640]) tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])
label:  b
image:  aslized_mj_bienvenu_7057 type and image: torch.Size([3, 360, 640]) tensor([[[0.0235, 0.0275, 0.0314,  ..., 0.0667, 0.0588

In [128]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        ##############################################################################
        # TODO: Design your own network, define layers here.                          #
        # Here We provide a sample of two-layer fc network from HW4 Part3.           #
        # Your solution, however, should contain convolutional layers.               #
        # Refer to PyTorch documentations of torch.nn to pick your layers.           #
        # (https://pytorch.org/docs/stable/nn.html)                                  #
        # Some common choices: Linear, Conv2d, ReLU, MaxPool2d, AvgPool2d, Dropout   #
        # If you have many layers, use nn.Sequential() to simplify your code         #
        ##############################################################################
        # from 28x28 input image to hidden layer of size 256
#         self.fc1 = nn.Linear(28*28, 8) 
        self.conv1 =nn.Conv2d(in_channels = 3,out_channels = 16,padding = 2, kernel_size = (5,5),stride = (2,2))
        self.pool =nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.conv2 =nn.Conv2d(in_channels = 16,out_channels = 64, padding = 2,kernel_size = (5,5),stride = (2,2))
        self.conv3 =nn.Conv2d(in_channels = 64,out_channels = 8, padding = 2,kernel_size = (5,5),stride = (2,2))
        self.fc_1 = nn.Linear(in_features = 8, out_features = 4) 
        
        self.init_weights()
        ##############################################################################
        #                             END OF YOUR CODE                               #
        ##############################################################################
    def init_weights(self):
        """Initialize all model parameters (weights and biases) in all layers to desired distributions"""

        torch.manual_seed(42)

        for conv in [self.conv1, self.conv2, self.conv3]:
            C_in = conv.weight.size(1)
            nn.init.normal_(conv.weight, 0.0, 1 / sqrt(5 * 5 * C_in))
            nn.init.constant_(conv.bias, 0.0)

        ## TODO: initialize the parameters for [self.fc_1]

        nn.init.normal_(self.fc_1.weight, 0.0, sqrt(1/self.fc_1.weight.size(1)))
        nn.init.constant_(self.fc_1.bias, 0.0)
        ##

    def forward(self, x):
        ##############################################################################
        # TODO: Design your own network, implement forward pass here                 # 
        ##############################################################################
        
        N, C, H, W = x.shape

        ## TODO: forward pass
        z= self.pool(F.relu(self.conv1(x)))
        z= self.pool(F.relu(self.conv2(z)))
        z = F.relu(self.conv3(z))

        # z=z.permute(*torch.arange(z.ndim - 1, -1, -1))

        z=torch.flatten(z, start_dim=1)

        # print("after resize: ",z.shape)
        z=self.fc_1(z)

        return z


model = Network().to(device)
criterion = nn.CrossEntropyLoss() # Specify the loss layer
print('Your network:')
print(summary(model, (3,28,28), device=device)) # visualize your model

##############################################################################
# TODO: Modify the lines below to experiment with different optimizers,      #
# parameters (such as learning rate) and number of epochs.                   #
##############################################################################
# Set up optimization hyperparameters
learning_rate = 1e-3
weight_decay = 1e-5
num_epoch = 20  # TODO: Choose an appropriate number of training epochs
optimizer = optim.Adam(model.parameters(), lr=learning_rate,
                       weight_decay=weight_decay) # Try different optimizers
##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

Your network:
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 14, 14]           1,216
         MaxPool2d-2             [-1, 16, 7, 7]               0
            Conv2d-3             [-1, 64, 4, 4]          25,664
         MaxPool2d-4             [-1, 64, 2, 2]               0
            Conv2d-5              [-1, 8, 1, 1]          12,808
            Linear-6                    [-1, 4]              36
Total params: 39,724
Trainable params: 39,724
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.04
Params size (MB): 0.15
Estimated Total Size (MB): 0.20
----------------------------------------------------------------
None


In [129]:
%%time
def train(model, trainloader, valloader, num_epoch=10):  # Train the model
    print("Start training...")
    trn_loss_hist = []
    trn_acc_hist = []
    val_acc_hist = []
    model.train()  # Set the model to training mode
    for i in range(num_epoch):
        running_loss = []
        print('-----------------Epoch = %d-----------------' % (i+1))
        for batch, label in tqdm(trainloader):
            batch = batch.to(device)
            label = label.to(device)
            optimizer.zero_grad()  # Clear gradients from the previous iteration
            # This will call Network.forward() that you implement
            pred = model(batch)
            loss = criterion(pred, label)  # Calculate the loss
            running_loss.append(loss.item())
            loss.backward()  # Backprop gradients to all tensors in the network
            optimizer.step()  # Update trainable weights
        print("\n Epoch {} loss:{}".format(i+1, np.mean(running_loss)))

        # Keep track of training loss, accuracy, and validation loss
        trn_loss_hist.append(np.mean(running_loss))
        trn_acc_hist.append(evaluate(model, trainloader))
        print("\n Evaluate on validation set...")
        val_acc_hist.append(evaluate(model, valloader))
    print("Done!")
    return trn_loss_hist, trn_acc_hist, val_acc_hist


def evaluate(model, loader):  # Evaluate accuracy on validation / test set
    model.eval()  # Set the model to evaluation mode
    correct = 0
    with torch.no_grad():  # Do not calculate grident to speed up computation
        for batch, label in tqdm(loader):
            batch = batch.to(device)
            label = label.to(device)
            pred = model(batch)
            correct += (torch.argmax(pred, dim=1) == label).sum().item()
        acc = correct/len(loader.dataset)
        print("\n Evaluation accuracy: {}".format(acc))
        return acc


trn_loss_hist, trn_acc_hist, val_acc_hist = train(model, trainloader,
                                                  valloader, num_epoch)

##############################################################################
# TODO: Note down the evaluation accuracy on test set                        #
##############################################################################
print("\n Evaluate on test set")
evaluate(model, testloader)

Start training...
-----------------Epoch = 1-----------------


  0%|                                                     | 0/4 [00:00<?, ?it/s]

image:  youtube_4_alex_abenchuchan_5403 type and image: torch.Size([3, 360, 640]) tensor([[[0.2000, 0.2000, 0.2000,  ..., 0.1843, 0.1843, 0.1843],
         [0.2000, 0.2000, 0.2000,  ..., 0.1843, 0.1843, 0.1843],
         [0.2000, 0.2000, 0.1961,  ..., 0.1843, 0.1843, 0.1843],
         ...,
         [0.1412, 0.1412, 0.1412,  ..., 0.1373, 0.1412, 0.1451],
         [0.1412, 0.1412, 0.1412,  ..., 0.1373, 0.1412, 0.1412],
         [0.1412, 0.1412, 0.1412,  ..., 0.1373, 0.1373, 0.1373]],

        [[0.1961, 0.1961, 0.1961,  ..., 0.1843, 0.1843, 0.1843],
         [0.1961, 0.1961, 0.1961,  ..., 0.1843, 0.1843, 0.1843],
         [0.1961, 0.1961, 0.1961,  ..., 0.1843, 0.1843, 0.1843],
         ...,
         [0.1412, 0.1412, 0.1373,  ..., 0.1333, 0.1373, 0.1373],
         [0.1373, 0.1373, 0.1373,  ..., 0.1333, 0.1333, 0.1333],
         [0.1373, 0.1373, 0.1373,  ..., 0.1333, 0.1294, 0.1294]],

        [[0.2000, 0.1961, 0.1922,  ..., 0.1843, 0.1843, 0.1843],
         [0.2000, 0.1961, 0.1922,  ..., 0

image:  misc_2_marlon_kuntze_2604 type and image: torch.Size([3, 360, 640]) tensor([[[0.1333, 0.1333, 0.1333,  ..., 0.1098, 0.1098, 0.1098],
         [0.1333, 0.1333, 0.1333,  ..., 0.1098, 0.1098, 0.1098],
         [0.1333, 0.1333, 0.1333,  ..., 0.1059, 0.1059, 0.1059],
         ...,
         [0.2196, 0.2196, 0.2196,  ..., 0.9961, 0.9961, 0.9961],
         [0.2196, 0.2196, 0.2196,  ..., 0.9961, 0.9961, 0.9961],
         [0.2196, 0.2196, 0.2196,  ..., 0.9961, 0.9961, 0.9961]],

        [[0.1373, 0.1373, 0.1373,  ..., 0.0863, 0.0863, 0.0863],
         [0.1373, 0.1373, 0.1373,  ..., 0.0863, 0.0863, 0.0863],
         [0.1373, 0.1373, 0.1373,  ..., 0.0863, 0.0863, 0.0863],
         ...,
         [0.2588, 0.2588, 0.2588,  ..., 0.9961, 0.9961, 0.9961],
         [0.2588, 0.2588, 0.2588,  ..., 0.9961, 0.9961, 0.9961],
         [0.2588, 0.2588, 0.2588,  ..., 0.9961, 0.9961, 0.9961]],

        [[0.1412, 0.1412, 0.1412,  ..., 0.0667, 0.0667, 0.0667],
         [0.1412, 0.1412, 0.1412,  ..., 0.0667,

image:  youtube_3_raymond_merritt_4774 type and image: torch.Size([3, 360, 640]) tensor([[[0.5294, 0.5294, 0.5294,  ..., 0.3608, 0.3647, 0.3686],
         [0.5294, 0.5294, 0.5294,  ..., 0.3608, 0.3647, 0.3686],
         [0.5333, 0.5333, 0.5333,  ..., 0.3647, 0.3647, 0.3686],
         ...,
         [0.6549, 0.6588, 0.6549,  ..., 0.4039, 0.4392, 0.4706],
         [0.6627, 0.6627, 0.6471,  ..., 0.4667, 0.4784, 0.4902],
         [0.6627, 0.6588, 0.6510,  ..., 0.4196, 0.4196, 0.4902]],

        [[0.3647, 0.3647, 0.3647,  ..., 0.3059, 0.3098, 0.3137],
         [0.3647, 0.3647, 0.3647,  ..., 0.3059, 0.3098, 0.3137],
         [0.3686, 0.3686, 0.3686,  ..., 0.3098, 0.3098, 0.3137],
         ...,
         [0.6510, 0.6510, 0.6235,  ..., 0.2627, 0.2941, 0.3294],
         [0.6549, 0.6392, 0.6078,  ..., 0.3059, 0.3176, 0.3255],
         [0.6471, 0.6275, 0.6078,  ..., 0.3137, 0.3137, 0.3098]],

        [[0.2157, 0.2157, 0.2157,  ..., 0.2627, 0.2667, 0.2706],
         [0.2157, 0.2157, 0.2157,  ..., 0.

  0%|                                                     | 0/4 [00:00<?, ?it/s]

image:  youtube_4_raymond_merritt_4839 type and image: torch.Size([3, 360, 640]) tensor([[[0.5294, 0.5294, 0.5294,  ..., 0.3608, 0.3647, 0.3686],
         [0.5294, 0.5294, 0.5294,  ..., 0.3647, 0.3686, 0.3686],
         [0.5333, 0.5333, 0.5333,  ..., 0.3647, 0.3686, 0.3686],
         ...,
         [0.6471, 0.6510, 0.6392,  ..., 0.4039, 0.4431, 0.4745],
         [0.6549, 0.6510, 0.6392,  ..., 0.4627, 0.4784, 0.4941],
         [0.6549, 0.6471, 0.6471,  ..., 0.4902, 0.4941, 0.4980]],

        [[0.3647, 0.3647, 0.3647,  ..., 0.3059, 0.3098, 0.3137],
         [0.3647, 0.3647, 0.3647,  ..., 0.3098, 0.3137, 0.3137],
         [0.3686, 0.3686, 0.3686,  ..., 0.3098, 0.3137, 0.3137],
         ...,
         [0.6549, 0.6549, 0.6275,  ..., 0.2627, 0.2980, 0.3255],
         [0.6549, 0.6471, 0.6196,  ..., 0.3137, 0.3176, 0.3255],
         [0.6471, 0.6353, 0.6235,  ..., 0.3216, 0.3176, 0.3098]],

        [[0.2157, 0.2157, 0.2157,  ..., 0.2627, 0.2667, 0.2706],
         [0.2157, 0.2157, 0.2157,  ..., 0.

AttributeError: 'tuple' object has no attribute 'to'